In [1]:
import os
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import re, os
from groq import Groq
import requests

In [2]:
client = Groq(
    api_key="******",  # This is the default and can be omitted
)

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Who are you?",
#         }
#     ],
#     model="llama3-8b-8192",
#     max_tokens=20
# )
# print(chat_completion.choices[0].message.content)

In [3]:
# load_dotenv("../local.env")

In [4]:
# client = InferenceClient(
#     api_key=os.environ["HF_TOKEN"],
# )

In [5]:
# completion = client.chat.completions.create(
#     model="openai/gpt-oss-120b",
#     messages=[
#         {
#             "role": "user",
#             "content": "What is the capital of France?"
#         }
#     ],
# )

In [6]:
## This is a tool
def calculate_expression(expression):
    safe_expr = re.sub(r'[^0-9+\-*/(). ]', '', expression)
    print(safe_expr)
    if safe_expr.strip() == "":
        return None
    try:
        return eval(safe_expr)
    except:
        return None

knowledge_base = {
    "python": "Python is a programming language known for its readability.",
    "openai": "OpenAI is the company that created ChatGPT.",
    "ai": "AI stands for Artificial Intelligence, which simulates human intelligence."
}

## This is a tool
#This is just SIMULATION for knowledge retrieval
def search_knowledge(query):
    q = query.lower()
    for keyword, value in knowledge_base.items():
        if keyword in q:
            return value
    return None

In [7]:
def query_openai_tool_selector(user_input):
    messages = [
        {
            "role": "system",
            "content": (
                "You're an AI assistant expert in understanding the user query and decide which tool would be"
                "appropriate to answer the user query. Based on the user's message, "
                "decide which tool to use: 'calculator', 'knowledge_search', or 'none'. "
                "Respond ONLY with a JSON object like this:\n"
                "{ \"tool\": \"calculator\", \"input\": \"5 * (4 + 3)\" }\n"
                "or\n"
                "{ \"tool\": \"none\", \"input\": \"Hello!\" }"
            )
        },
        {"role": "user", "content": user_input}
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192")
    try:
        tool_call = eval(response.choices[0].message.content)
        return tool_call.get("tool"), tool_call.get("input")
    except Exception as e:
        return "none", user_input

def query_openai(user_input):
    tool, tool_input = query_openai_tool_selector(user_input)
    print(tool,"--->", tool_input)
    if tool == "calculator":
        result = calculate_expression(tool_input)
        return f"The result is: {result}" if result else "I couldn't compute that."
    elif tool == "knowledge_search":
        result = search_knowledge(tool_input)
        return result if result else "I couldn't find relevant information."
    else:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_input}
            ],
            model="llama3-8b-8192")
        
        return response.choices[0].message.content
print("Welcome! Ask me anything. Type 'quit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Agent: Goodbye!")
        break
    print("Agent:", query_openai(user_input))

Welcome! Ask me anything. Type 'quit' to stop.
You: What is openai


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [8]:
from sympy import symbols, Eq, solve, sympify

def solve_equation(equation: str):
    """
    Solves a given algebraic equation and returns the solution(s).

    Args:
        equation (str): An equation as a string, e.g., "2*x + 5 = 15"

    Returns:
        dict: {
            "equation": "<original equation>",
            "solutions": [list of solutions as strings]
        }
    """
    try:
        # Split equation into LHS and RHS
        if "=" not in equation:
            return {
                "equation": equation,
                "error": "Equation must contain '=' sign"
            }
        
        lhs, rhs = equation.split("=")
        lhs = sympify(lhs.strip())
        rhs = sympify(rhs.strip())
        
        # Detect variables
        vars_in_eq = list(lhs.free_symbols.union(rhs.free_symbols))
        if not vars_in_eq:
            return {
                "equation": equation,
                "error": "No variable found in equation"
            }
        
        # Assume solving for the first variable
        var = vars_in_eq[0]
        solutions = solve(Eq(lhs, rhs), var)
        
        return {
            "equation": equation,
            "solutions": [str(s) for s in solutions]
        }
    
    except Exception as e:
        return {
            "equation": equation,
            "error": str(e)
        }


In [12]:
solve_equation("3*x + 2 = 32")

{'equation': '3*x + 2 = 32', 'solutions': ['10']}